In [1]:
%pip install torch torchvision numpy plotly sympy pandas nbformat

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from dataclasses import dataclass
import plotly.express as px
import numpy as np
import pandas as pd

In [22]:
@dataclass
class Product:
    name: str
    icon: str
    decay_per_day: float
    demand_mean: float
    demand_variance: float

TOMATO = Product("tomato", "🍅", 0.23, 2, 1)
SALAD = Product("salad", "🥬", 0.1, 1, 4)
EGGPLANT = Product("eggplant", "🍆", 0.1, 1, 3)
PEACH = Product("peach", "🍑", 0.5, 0.2, 0.04)

products = [TOMATO, SALAD, EGGPLANT, PEACH]

In [26]:
SAMPLES = 1024

df_dict = dict()

df_dict['timestamp'] = list(range(SAMPLES))

for i, product in enumerate(products):
    from math import sqrt
    samples = np.random.default_rng().lognormal(product.demand_mean, sqrt(product.demand_variance), SAMPLES).astype(int)
    df_dict[product.name + '_demand'] = samples
    df = pd.DataFrame(df_dict)
    fig = px.line(df, x="timestamp", y=product.name + "_demand", title=product.icon)
    fig.show()

df = pd.DataFrame(df_dict)
df

,timestamp,tomato_demand,salad_demand,eggplant_demand,peach_demand
0,0,3,5,8,1
1,1,2,25,3,1
2,2,18,1,5,1
3,3,3,1,0,0
4,4,7,0,16,1
...,...,...,...,...,...
1019,1019,10,0,4,0
1020,1020,1,5,0,0
1021,1021,3,1,0,0
1022,1022,8,0,0,1


In [ ]:
# ins: 
#   1. day
#   2. month
#   3. year
#               bought / wasted
#   4-7 tomatoes
#   8-11 salad
#   12-14 eggplants
#   15-18 boxes
# outs:
#   1-4 optmized buy for each of the 4 classes

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# hyperparameters

input_size = 18
sequence_length = 7
num_layers = 5
num_classes = 4
hideen_size = 256
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagation
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

model = RNN(input_size, hideen_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(...):
        ...